In [12]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import math

import importlib
import data_analysis
importlib.reload(data_analysis)

<module 'data_analysis' from '/home/jlimonet/Documents/Stage_M2/code/data_analysis.py'>

In [21]:
device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

/home/jlimonet/Documents/Stage_M2/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [16]:
from data_analysis import Simulation
from data_analysis import compatible_path
current_directory = compatible_path('../')

simulation = Simulation(current_directory, normalize = True)
time, x, z, u, w, T, umean, wmean, Tmean = simulation.import_data()
simulation.image_rgb()

h, l = np.shape(x)
m = len(time)
print(m)
N = h*l*3

6249


In [14]:
print(np.max(simulation.X), np.min(simulation.X))
print(simulation.normalize)

1.0 -1.0
True


In [4]:
print(np.shape(simulation.X))

(12393, 6249)


In [10]:
6 * 1024 * 1024 * 1024//(12393*4)

129961

In [5]:
class SimuDataset(Dataset):

    def __init__(self, simu, rgb = False, transform = None):
        X = simu.X
        if rgb : 
            X = simu.X_rgb
        self.x = torch.from_numpy(X)
        self.n_snapshots = X.shape[1]
        self.transform = transform
        
    def __getitem__(self, index):
        sample = self.x[:,index]
        
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return self.n_snapshots
    
    def plot_field(self, t, save=False, directory=None):
        fig, ax = plt.subplots(figsize=(15, 5))
        vmin = self.T[t, :, :].min()
        vmax = self.T[t, :, :].max()
        abs_max = max(abs(vmin), abs(vmax))
        ax.streamplot(
            self.x.T,
            self.z.T,
            self.u[t, :, :].T,
            self.w[t, :, :].T,
            color="k",
            arrowsize=0.7,
            linewidth=1,
        )
        levels = np.linspace(vmin, vmax, 20)
        cf0 = ax.contourf(
            self.x,
            self.z,
            self.T[t, :, :],
            levels=levels,
            cmap=cm.Spectral.reversed(),
            norm=matplotlib.colors.Normalize(vmin=vmin, vmax=vmax),
        )
        cbar = plt.colorbar(cf0, ax=ax, shrink=0.35, aspect=6, ticks= [-abs_max, 0, abs_max])
        cbar.ax.set_aspect("auto")
        ax.set_title(f"Temperature and velocity field at t = {t}")
        ax.set_aspect("equal")
        ax.set_ylim(0, 1)
        ax.set_xlim(-4, 4)


        if save:
            plt.savefig(directory, dpi=300)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()

In [18]:
class LinearAutoencoder(nn.Module):
    def __init__(self, n_channels, N):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_channels*N, 128*16), 
            nn.ReLU(), 
            nn.Linear(128*16, 64*16), 
            nn.ReLU(), 
            nn.Linear(64*16, 12*16))
        
        self.decoder = nn.Sequential(
            nn.Linear(12*16, 64*16), 
            nn.ReLU(), 
            nn.Linear(64*16, 128*16), 
            nn.ReLU(), 
            nn.Linear(128*16, n_channels*N),
            nn.Tanh())
        
    def forward(self, x):
        # print(f'size of x {x.shape}')
        x = x.float()
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class SimpleLinearAutoencoder(nn.Module):
    def __init__(self, N):
        super().__init__()
        self.encoder = nn.Linear(N, 128)
        
        self.decoder = nn.Sequential(
            nn.Linear(128, N), 
            nn.Tanh())
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [15]:
batch_size = 4

dataset = SimuDataset(simulation)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)

tensor(-0.6515, dtype=torch.float64) tensor(0.5522, dtype=torch.float64) torch.Size([4, 12393])


In [19]:
batch_size = 4


model = SimpleLinearAutoencoder(N)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

In [8]:
print(batch_size*n_channels*N, 12393*4)
print(N)

49572 49572
4131


In [20]:
num_epoch = 1
output = []
for epoch in range(num_epoch):
    for map in dataloader:
        map = map.float()
        reconstructed = model(torch.flatten(map))
        loss = criterion(reconstructed, torch.flatten(map))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss:{loss.item() : .4f}")
    output.append((epoch, map, reconstructed))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x49572 and 12393x128)

In [10]:
# training loop
num_epoch = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)

print(total_samples, n_iterations)

for epoch in range(num_epoch):
    for i, inputs in enumerate(dataloader):
        #forward, backward, update weigths
        if (i+1)%5 == 0 :
            print(f'epoch = {epoch + 1}, step {i+1}/{n_iterations}, inputs {inputs.shape}')

6249 1563
epoch = 1, step 5/1563, inputs torch.Size([4, 12393])
epoch = 1, step 10/1563, inputs torch.Size([4, 12393])
epoch = 1, step 15/1563, inputs torch.Size([4, 12393])
epoch = 1, step 20/1563, inputs torch.Size([4, 12393])
epoch = 1, step 25/1563, inputs torch.Size([4, 12393])
epoch = 1, step 30/1563, inputs torch.Size([4, 12393])
epoch = 1, step 35/1563, inputs torch.Size([4, 12393])
epoch = 1, step 40/1563, inputs torch.Size([4, 12393])
epoch = 1, step 45/1563, inputs torch.Size([4, 12393])
epoch = 1, step 50/1563, inputs torch.Size([4, 12393])
epoch = 1, step 55/1563, inputs torch.Size([4, 12393])
epoch = 1, step 60/1563, inputs torch.Size([4, 12393])
epoch = 1, step 65/1563, inputs torch.Size([4, 12393])
epoch = 1, step 70/1563, inputs torch.Size([4, 12393])
epoch = 1, step 75/1563, inputs torch.Size([4, 12393])
epoch = 1, step 80/1563, inputs torch.Size([4, 12393])
epoch = 1, step 85/1563, inputs torch.Size([4, 12393])
epoch = 1, step 90/1563, inputs torch.Size([4, 12393])
e

In [ ]:
class ConvolutionalAutoencoder(nn.Module):

    def __init__(self, batch_size, n_channels, N):
        super().__init__()
        self.batch_size = batch_size
        #shape : B * 3 * 81 * 51
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 5), #shape : B * 16 * 75 * 45
            nn.ReLU(), 
            nn.Conv2d(16, 32, 5), #shape : B * 32 * 81 * 51
            nn.ReLU(), 
            nn.Conv2d(32, 64, 5))
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64,32,5), 
            nn.ReLU(), 
            nn.ConvTranspose2d(32,16,5), 
            nn.ReLU(), 
            nn.Linear(16,3,5),
            nn.Tanh())
        
    def forward(self, x):
        print(f'size of x {x.shape}')
        encoded = self.encoder(torch.reshape(x, (self.batch_size*N)))
        print(1)
        print(encoded.shape)
        decoded = self.decoder(encoded)
        return(torch.reshape(decoded, (self.batch_size, N)))


In [73]:
81-5

76

In [59]:
import torch.nn.functional as F
class ConvolutionalAutoencoder(nn.Module):

    def __init__(self, batch_size, n_channels, N):
        super().__init__()
        self.batch_size = batch_size
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)
        self.conv3 = nn.Conv2d(12, 28, 5)

    def forward(self, x):
        x = self.conv1(x)
        print('Shape after conv1:', x.shape)
        x = F.relu(x)
        x = self.conv2(x)
        print('Shape after conv2:', x.shape)
        x = F.relu(x)
        x = self.conv3(x)
        print('Shape after conv3:', x.shape)
        return x

In [72]:
batch_size = 4
n_channels = 3


dataset = SimuDataset(simulation, rgb = True)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)
model = ConvolutionalAutoencoder(batch_size, n_channels, N)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

tensor(-0.7316, dtype=torch.float64) tensor(0.7301, dtype=torch.float64) torch.Size([4, 3, 81, 51])


In [49]:
print(np.shape(simulation.X_rgb))
print(np.shape(simulation.u))
print(4*3*81*51)
print(4*6*40*25)

(3, 6249, 81, 51)
(6249, 81, 51)
49572
24000


In [63]:
num_epoch = 1
output = []
for epoch in range(num_epoch):
    for map in dataloader:
        map = map.float()
        print(f'map shape : {map.shape}')
        reconstructed = model(map)
        print(f'reconstructed shape : {reconstructed.shape}')
        loss = criterion(reconstructed, map)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss:{loss.item() : .4f}")
    output.append((epoch, map, reconstructed))

map shape : torch.Size([4, 3, 81, 51])
Shape after conv1: torch.Size([4, 6, 40, 25])
Shape after conv2: torch.Size([4, 12, 19, 12])
Shape after conv3: torch.Size([4, 28, 15, 8])
reconstructed shape : torch.Size([4, 28, 15, 8])


/home/julielimonet/Documents/Stage_M2/code/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 3, 81, 51])) that is different to the input size (torch.Size([4, 28, 15, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (51) at non-singleton dimension 3